 # Solvation free energy of ethanol
![ethanol-water.png](./images/ethanol-water.png)

 ## Background
In this tutorial, we’ll calculate the free energy of solvation of a small molecule: ethanol. This type of calculation can either be done on its own, or can be part of a binding free energy calculation. Such calculations can be important, because the free energy is the most important static quantity in a thermal system: its sign determines the whether of a molecule will be soluble, or whether it will bind to another molecule.

We will start this tutorial with some background on how to calculate free energies, and how a free energy of solvation relates to a free energy of binding calculation. Then, we will focus on the practicalities of doing such a calculation in GROMACS. You should use GROMACS 2019 for this tutorial.
 ## Calculating a binding free energy

Calculating free energies can usually only be done using small steps and a full path between one end state and the other. For example, to calculate the binding free energy of the ligand to a protein, we ultimately need to compare the situation of the ligand being bound to the protein, to the situation where both the ligand and the protein are separately in solution:

![ligand-binding.png](./images/ligand-binding.png)

This could be calculated directly, for example by dragging the ligand away from the protein and integrating the potential of mean force (averaging the force, and integrating it). Forces have very large fluctuations, however, and this turns out to be much more expensive than using *free energy perturbation* methods such as the Bennett Acceptance Ratio (BAR) that we’ll use in the tutorial.

Remember that a free energy difference between two states A and B determines their relative probability $p_A$ and $p_B$,
$$ \frac{p_A}{p_b} = exp \frac{F_B - F_A}{k_BT}$$
where $k_B$ is Boltzmann’s constant relating thermal energy to the temperature ($1.38·10^{-23}$ J/K), and T is the temperature. We could, in principle, calculate a free energy difference by waiting long enough, and measuring how often the system is in which state. The free energy differences, however, are often of the order of tens of kJ/mol: for example, the free energy of solvation of ethanol at 298K is -20.1 kJ/mol, which is equivalent to -8.1 $k_BT$: a relative probability of $3·10^{-4}$. We would need to wait a long time for that transition to occur spontaneously, and even longer to get good statistics on it.
Because of this probability issue, free energy methods rely on one basic idea: to force the system to where it doesn’t want to be, and then measure by how much it doesn’t want to be there. In free energy perturbation methods, we force the system by coupling the interaction strength between a molecule of interest and the rest of the system to a variable $\lambda$:
$$ E_{total} = E_{ligand-ligand} + E_{rest-rest} + \lambda E_{ligand-rest}  $$
and we slowly turn $\lambda$ from 1 to 0. This means we can effectively turn off a molecule, and pretend that it is in vacuum (at $\lambda=0$): we force the system to where it doesn’t want to be (either in the solvated or in the vacuum state, depending on what the sign of the free energy difference is). We’ll then use the BAR method of calculating by how much it doesn’t want to be there.

Coupling and de-coupling in this way helps us with calculating the free energy of binding, because we can now create a two-step path:
![fep.png](./images/fep.png)
where we first de-couple the ligand from the solvent, and then re-solvate the ligand in the presence of the protein. The free energy of binding is thus 
$$ \Delta G_{binding} = \Delta G_1 + \Delta G_2  $$
and the simulation is split into two parts: one calculating the de-solvation free energy, and one involving the free energy of coupling of a molecule into the system with a protein. That last simulation couples the ligand from $\lambda=0¤ where it doesn’t interact with the system, to the situation at $\lambda=1$, where the protein is bound to the ligand. The first simulation is the inverse of a free energy of solvation. This is the one we’ll concentrate on in this tutorial - partly for computational performance reasons: because there is no protein involved, the simulation box size can be small and the simulations will be fast.

 ## Free energy of solvation

To calculate a free energy of solvation, we calculate $-\Delta G_1$ in the picture above, or, equivalently, $\Delta G_{solv}$ in this picture:
![fe-solvation.png](./images/fe-solvation.png)
We’ll do this coupling our molecule to a variable $\lambda$ (see equations above) and Bennett Acceptance Ratio (BAR) calculations, as built into GROMACS.

The BAR method relies on the output of pairs of simulations, say at state $\lambda$A and $\lambda$B. The free energy difference can be calculated directly if $\lambda$A and $\lambda$B are close enough (see Bennett’s original article: Bennett, J. Comp. Phys, (1976) vol. 22 p. 245 for details), by calculating the Monte Carlo acceptance rates of transitions from $\lambda$A to $\lambda$B and vice versa, mapping states from $\lambda$A and $\lambda$B. The term ‘close enough’ here means that switching between the two states should be possible in both directions: some of the same configurations should be allowed in both end points (i.e. they should share some parts of phase space).

The most obvious points for $\lambda$A and $\lambda$B would be $\lambda$A =0 and $\lambda$B=1. These end points, however, usually have very few states in common: they share very little phase space. Because of this, the free energy would never converge to a usable value. That’s why we’ll split up the problem:
![fep-coupling.png](./images/fep-coupling.png)
with as many $\lambda$ points as are needed. We will therefore effectively ‘slowly’ turn on (or off) the interactions between our ligand and the solvent. This means that we need to run as many simulations as there are $\lambda$ points, that we need to tell each simulation which neighboring $\lambda$ points there are, and that we will post-process the results combining the results of many simulations (we will use 7 $\lambda$ points: 0, 0.2, 0.4, 0.6, 0.8, 0.9 and 1). As an example, we will run one simulation at $\lambda$=0.4, and that simulation will calculate the energy differences between its $\lambda$ point and the neighboring points $\lambda$=0.2 and $\lambda$=0.6.

We will take one shortcut: we will turn off both the electrostatic (Coulomb) interactions and the Van der Waals (Lennard-Jones) interactions at the same time. For high-quality results, these stages are normally separated, but here we will do them both at the same time for expediency. Gromacs uses ‘soft-core’ interactions to make sure that while the normal (Lennard-Jones and Coulomb) interactions are being turned off, there will never be two point charges sitting on top of each other: this is achieved by turning on an interaction that effectively repels particles at intermediate $\lambda$ points (in such a way that it cancels out from the free energy difference).


 ## Preparing the system
Look for a file named **topol.top**, and a very basic coordinate file named **ethanol.gro**. This topology uses the OPLS force field and defines a methane molecule, and includes the definitions for SPC/E water.

We will use the following 'boilerplate' code to prepare some files.


In [ ]:
import shlex,os,modulecmd,subprocess,shutil

#allow use of modules
module=modulecmd.Modulecmd()

#load gromacs module
module.load('gromacs/2019')

#simple terminal functionality
def terminal_call(command_string, directory):
    command=shlex.split(command_string)
    proc=subprocess.run(command,cwd=directory,stdout=subprocess.PIPE,stderr=subprocess.STDOUT)
    print(proc.stdout.decode())

#simple function to write mdp files
def write_mdp(mdp_string, mdp_name, directory):
    mdp_filename=os.path.join(directory,mdp_name)
    mdp_filehandle=open(mdp_filename,'w')
    mdp_filehandle.write(mdp_string)
    mdp_filehandle.close()

#get the path to the working directory
pwd=os.getcwd()

#path to gromacs files
input_files=os.path.join(pwd,'input_files')
output_files=os.path.join(pwd,'output_files')
if not os.path.isdir(output_files):
    os.mkdir(output_files)
shutil.copy(os.path.join(input_files,'ethanol.gro'),os.path.join(output_files,'ethanol.gro'))
shutil.copy(os.path.join(input_files,'topol.top'),os.path.join(output_files,'topol.top'))

**Question**: Take a look at the topology file topol.top. For the ethanol molecule definition, can you find which atoms are there, and how they are connected? We got the ethanol parameters by borrowing a threonine side chain...
We will first prepare the simulation box: the original configuration file has a dummy simulation box associated with it (you can see that by looking at the file **ethanol.gro**). We do this with the following cell.


In [ ]:
editconf_string="gmx editconf -f ethanol.gro -o box.gro -bt dodecahedron -d 1"
terminal_call(editconf_string,output_files)

This sets up the simulation box. In this case, it will make the simulation box a rhombic dodecahedron with a minimum distance between the solute (the ethanol molecule) and the box edge of 1nm. The box is a rhombic dodecahedron because it provides a more effective packing of periodic images than rectangular boxes: we can use fewer waters for the same distance between periodic images of the ethanol molecule. See the Gromacs manual for illustrations of this box shape and how its periodic images are arranged.

Next, we solvate the system in water with the following cell.


In [ ]:
solvate_string="gmx solvate -cp box.gro -cs -o solvated.gro -p topol.top"
terminal_call(solvate_string,output_files)

This should generate a system with ~300 water molecules taken from the default file name of the -cs option: a box of equilibrated water molecules. To make the configuration suitable for simulation, we will first minimize its energy, twice: once with flexible bonds, and once with constrained bonds. For the minimization we will use the following settings (see the included file em.mdp)

We will create the mdp file by executing the next cell.


In [ ]:
em_mdp=""";minimal mdp options for energy minimization
integrator               = steep
nsteps                   = 500
coulombtype              = pme
vdw-type                 = pme
"""
write_mdp(em_mdp,'em.mdp',output_files)

Prepare for the minimization by preprocessing the input files into a run file with the following cell.


In [ ]:
em_grompp_string='gmx grompp -f em.mdp -c solvated.gro -o em.tpr'
terminal_call(em_grompp_string,output_files)

This generates the file **em.tpr**. We will run the mimimization with the following cell.


In [ ]:
em_md_string='gmx mdrun -v -deffnm em -ntmpi 1'
terminal_call(em_md_string,output_files)

Here we have had to set **-ntmpi 1** to specify only using one mpi rank since the system is so small. For larger systems more mpi ranks would be appropriate. For very small systems such as this one, it would also be possible to use **-nt 1** to specify only using one processor.

**Question:** what happens if you change the **-nt** or **-ntmpi** to different numbers? What would do you think would happen on a larger system?



 ## Global equilibration
We are now ready to equilibrate the system thermally. For this we will turn on pressure and temperature coupling: we’re trying to calculate the difference in Gibbs free energy, and for that, the system must maintain temperature, but also pressure, while the ethanol molecule is de-coupled. The global equilibration (i.e. the equilibration done before we impose several different values) is done with **equil.mdp**:


In [ ]:
equil_mdp=""";equilibration mdp options
integrator               = md
nsteps                   = 20000
dt                       = 0.002
nstenergy                = 100
rlist                    = 1.0
nstlist                  = 10
vdw-type                 = pme
rvdw                     = 1.0
coulombtype              = pme
rcoulomb                 = 1.0
fourierspacing           = 0.12
constraints              = all-bonds
tcoupl                   = v-rescale
tc-grps                  = system
tau-t                    = 0.2
ref-t                    = 300
pcoupl                   = berendsen
ref-p                    = 1
compressibility          = 4.5e-5
tau-p                    = 5
gen-vel                  = yes
gen-temp                 = 300
"""
write_mdp(equil_mdp,'equil.mdp',output_files)

We’ll be using the v-rescale thermostat, and the Berendsen barostat. We run equilibration in the following cell.


In [ ]:
equil_grompp_string='gmx grompp -f equil.mdp -c em.gro -o equil.tpr'
terminal_call(equil_grompp_string,output_files)
equil_md_string='gmx mdrun -deffnm equil -v -ntmpi 1'
terminal_call(equil_md_string,output_files)

As before, you might need to limit the number of cores used. After this we should be ready with a *hopefully* equilibrated configuration of ethanol in water in a minute. The name of the output configuration is **equil.gro**. 

**Question**: You should check whether the system has been equilibrated. How could you do this?


## Creating the $\lambda$ points
After the equilibration is done, we are ready to split the system into different $\lambda$ points. We will do this in the following cell, which will run **grompp** and **mdrun** sequntially for each $\lambda$ point. We could also create slurm submit scripts for this if the jobs were slower, but for a system of this size it is not necessary.

The free energy settings state the following: take the molecule ethanol, and couple it to our variable $\lambda$. This is done so that $\lambda=0$ means that the molecule is de-coupled, and $\lambda=1$ means that the molecule is fully coupled (vdwq means LennardJones + Coulomb). The **sc-power**, **sc-sigma** and **sc-alpha** settings control the ‘soft-core’ interactions that prevent the system from having overlapping particles as it is de-coupled.

The only things that still need to be set are the actual $\lambda$ value **init-lambda** and the **foreign-lambda** value: that field determines for which other $\lambda$ values the simulation should calculate energy differences. For our purposes, we will just calculate energy differences to all other $\lambda$ values and keep this the same for all simulations (the performance impact of this is negligible).


In [ ]:
run_mdp="""; we'll use the sd integrator with 100000 time steps (200ps)
integrator               = sd
nsteps                   = 100000
dt                       = 0.002
nstenergy                = 1000
nstlog                   = 5000
; cut-offs at 1.0nm
rlist                    = 1.0
dispcorr                 = EnerPres
vdw-type                 = pme
rvdw                     = 1.0
; Coulomb interactions
coulombtype              = pme
rcoulomb                 = 1.0
fourierspacing           = 0.12
; Constraints
constraints              = all-bonds
; set temperature to 300K
tcoupl                   = v-rescale
tc-grps                  = system
tau-t                    = 0.2
ref-t                    = 300
; set pressure to 1 bar with a thermostat that gives a correct
; thermodynamic ensemble
pcoupl                   = parrinello-rahman
ref-p                    = 1
compressibility          = 4.5e-5
tau-p                    = 5

; and set the free energy parameters
free-energy              = yes
couple-moltype           = ethanol
; these 'soft-core' parameters make sure we never get overlapping
; charges as lambda goes to 0
sc-power                 = 1
sc-sigma                 = 0.3
sc-alpha                 = 1.0
; we still want the molecule to interact with itself at lambda=0
couple-intramol          = no
couple-lambda1           = vdwq
couple-lambda0           = none
init-lambda-state        = {}
; These are the lambda states at which we simulate
; for separate LJ and Coulomb decoupling, use
fep-lambdas              = 0.0 0.2 0.4 0.6 0.8 0.9 1.0
"""

We will actually run the simulations in the following cell.


In [ ]:
number_of_lambdas=7
for lambda_number in range(number_of_lambdas):
    lambda_directory=os.path.join(output_files,'lambda_{:0>2}'.format(lambda_number))
    os.mkdir(lambda_directory)
    gro_file=os.path.join(output_files,'equil.gro')
    top_file=os.path.join(output_files,'topol.top')
    shutil.copy(gro_file,os.path.join(lambda_directory,'conf.gro'))
    shutil.copy(top_file,lambda_directory)
    write_mdp(run_mdp.format(lambda_number),'grompp.mdp',lambda_directory)
    run_grompp_string='gmx grompp'
    terminal_call(run_grompp_string,lambda_directory)
    run_mdrun_string='gmx mdrun -nt 1'
    terminal_call(run_mdrun_string,lambda_directory)

 ## Post-processing: extracting the free energy
After the simulations are done, we can extract the full free energy difference from the output data. Check your directories lambda_00 to lambda_06 for files called dhdl.xvg. These contain the energy differences that are going to be used to calculate the free energy difference. Combine them into a free energy with the Gromacs BAR tool gmx bar in the following cell.


In [ ]:
for lambda_number in range(number_of_lambdas):
    lambda_directory=os.path.join(output_files,'lambda_{:0>2}'.format(lambda_number))
    bar_string='gmx bar -b 100 -f dhdl.xvg'
    terminal_call(bar_string,lambda_directory)

This is pure magic. As part of the free energy code in mdrun, we have already calculated the offset enthalpies to the adjacent $\lambda$ points, so this is already available in the dhdl.xvg file (together with all information about the point itself, what simulation it was, etc) - no need to rerun any simulations or store the entire trajectories. Second, the gmx bar command will do all the complicated processing needed for Bennett Acceptance Ratio free energies and just give you the results.

Where the -b 100 means that the first 100 ps should be disregarded: they serve as another equilibration, this time at the conditions of the simulation. You should get a free energy difference of approximately -20.6 +/- 2.4 kJ/mol (this may be different if you run on different hardware: this answer is from a standard x86_64 cluster). This should be compared to an experimental value of -20.9 kJ/mol.

**Question**: Longer runs will change the free energy value a bit as the standard error estimate shrinks (try it). Exactly what value you get will depend on a number of settings, such as whether you use LJ-PME. Why can there sometimes be a significant (i.e. bigger than the estimated error) difference between the experimental result and the simulation result? How could this be improved?

**Question**: Look at the error bars for the individual $\lambda$ points: they vary a lot between individual point pairs. What does this mean for the efficiency for the overall calculation? How could it be improved?


 ## Where to go from here
After calculating the free energy of solvation, we’ve solved the first part of the free energy of binding of the earlier equations. The second part involves coupling a molecule into (or out of) a situation where it is bound to a protein. This introduces one additional complexity: we end up with a situation where a weakly coupled ligand wanders through our system:
![weak-couple.png](./images/weak-couple.png)
which is bad because this is a poorly reversible situation: there are suddenly very few states that map from a weakly coupled to a more strongly coupled molecule, which will drastically reduce the accuracy of the free energy calculation.

This situation can be remedied by forcing the ligand to stay at a specific position relative to the protein. This can be done with the Gromacs ‘pull code’, which allows the specification of arbitrary forces or constraints onto with respect to centers of mass of any chosen set of atoms onto any other group of atoms. With a pull type of ‘umbrella’, we can specify that we want a quadratic potential to this specified location, forcing the ligand to stay at its native position even when it has been fully de-coupled.

One way find out where to put the center of the force is by choosing a group of atoms in the protein close to the ligand, and doing a simulation with full ligand coupling, where the pull code is enabled, but with zero force. The pull code will then frequently output the coordinates of the ligand, from which an average position and an expected deviation can be calculated. This can then serve as a reference point for the location of the center of force for the pull code during the production runs, and the force constant of the pull code.

Once the free energy has been calculated, care must be taken to correct for the fact that we have trapped our molecule. This can easily be done analytically.

**Optional Question**: Given a measured standard deviation in the location of the center of mass of our ligand, how do we choose the force constant for the pull code?

**Optional Question**: How do we correct for using the pull code: what is the contribution to the free energy of applying a quadratic potential to a molecule?
